# DX 704 Week 1 Project

This week's project will build a portfolio risk and return model, and make investing recommendations for hypothetical clients.
You will collect historical data, estimate returns and risks, construct efficient frontier portfolios, and sanity check the certainty of the maximum return portfolio.

The full project description and a template notebook are available on GitHub at the following link.

https://github.com/bu-cds-dx704/dx704-project-01


Feel free to use optimization tools or libraries (such as CVXOPT or scipy.optimize) to perform any calculations required for this mini project.

### Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Data

Collect historical monthly price data for the last 24 months covering 6 different stocks.
The data should cover 24 consecutive months including the last month that ended before this week's material was released on Blackboard.
To be clear, if a month ends between the Blackboard release and submitting your project, you do not need to add that month.

The six different stocks must include AAPL, SPY and TSLA.
At least one of the remaining 3 tickers must start with the same letter as your last name (e.g. professor Considine could use COIN).
This is to encourage diversity in what stocks you analyze; if you discuss this project with classmates, please make sure that you pick different tickers to differentiate your work.
Do not pick stocks with fewer than 24 consecutive months of price data.

In [4]:
# --- Part 1: Build and save historical_prices.tsv (24 months, last trading day) ---

import sys
import pandas as pd

# Optional: auto-install yfinance if missing (works in Jupyter/Colab)
try:
    import yfinance as yf
except ImportError:
    print("yfinance not found; installing...")
    !{sys.executable} -m pip install -q yfinance
    import yfinance as yf

# Your six tickers
tickers = ["AAPL", "SPY", "TSLA", "MSFT", "NVDA", "PLTR"]

# Compute window: last 24 COMPLETE months
today = pd.Timestamp.today().tz_localize(None)
first_of_this_month = pd.Timestamp(today.year, today.month, 1)
end_month = first_of_this_month - pd.Timedelta(days=1)                # last calendar day of previous month
start_month = (end_month - pd.DateOffset(months=23)).replace(day=1)   # first day 23 months earlier (24 months total)

# Download enough daily data to cover the window (with a small buffer)
df = yf.download(
    tickers=tickers,
    start=start_month - pd.Timedelta(days=3),
    end=end_month + pd.Timedelta(days=1),      # yfinance end is exclusive
    progress=False,
    auto_adjust=False,                          # keep explicit Adj Close
    group_by="column"
)

# Extract Adjusted Close regardless of df shape (MultiIndex expected)
if isinstance(df.columns, pd.MultiIndex):
    adj = df["Adj Close"].copy()
else:
    # Fallback: single-level columns (rare). Try to keep only our tickers.
    adj = df.filter(items=tickers).copy()

# Restrict to our monthly window (safety slice)
adj = adj.loc[start_month:end_month]

# Resample to month-end and take the last available TRADING day in each month
monthly = adj.resample("M").last()

# Keep exactly 24 rows and ensure column order
monthly = monthly.loc[(monthly.index >= start_month) & (monthly.index <= end_month)]
monthly = monthly.tail(24).reindex(columns=tickers)

# Final table: 'date' + 6 tickers
monthly.index.name = "date"
prices_wide = monthly.reset_index()

# Optional sanity checks
assert prices_wide.shape[0] == 24, f"Expected 24 months, got {prices_wide.shape[0]}"
assert list(prices_wide.columns) == ["date"] + tickers

# Save as TSV
prices_wide.to_csv("historical_prices.tsv", sep="\t", index=False)

print(f"Saved historical_prices.tsv with shape {prices_wide.shape}")
print(f"Window: {prices_wide.iloc[0,0].date()} → {prices_wide.iloc[-1,0].date()}")
display(prices_wide.head())


Saved historical_prices.tsv with shape (24, 7)
Window: 2023-09-30 → 2025-08-31


/tmp/ipykernel_5870/1651105955.py:44: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly = adj.resample("M").last()


Ticker,date,AAPL,SPY,TSLA,MSFT,NVDA,PLTR
0,2023-09-30,169.549286,417.865631,250.220001,311.062347,43.475830,16.000000
1,2023-10-31,169.113541,408.794403,200.839996,333.090363,40.758278,14.800000
2,2023-11-30,188.355331,446.135193,240.080002,374.042236,46.745087,20.049999
3,2023-12-31,190.913666,466.503662,248.479996,371.209167,49.499973,17.170000
4,2024-01-31,182.851913,473.933472,187.289993,392.472443,61.499634,16.090000


Save the data as a TSV file named "historical_prices.tsv" and include a header row with the column names "date" and the 6 stock ticker symbols.
The date should be the last trading day of the month, so it may not be the last day of the month.
For example, the last trading day of November 2024 was 2024-11-29.
The remaining columns should contain the adjusted closing prices of the corresponding stock tickers on that day.


In [5]:
# YOUR CHANGES HERE


import yfinance as yf
import pandas as pd

tickers = ["AAPL", "SPY", "TSLA", "MSFT", "NVDA", "PLTR"]

df = yf.download(tickers, period="26mo", interval="1d", auto_adjust=False, progress=False)

adj = df["Adj Close"]

monthly = adj.resample("M").last()

monthly = monthly.tail(24)

monthly = monthly.reset_index().rename(columns={"Date": "date"})

monthly.to_csv("historical_prices.tsv", sep="\t", index=False)

print("Saved historical_prices.tsv")
display(monthly.head())


Saved historical_prices.tsv


/tmp/ipykernel_5870/2791013402.py:13: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly = adj.resample("M").last()


Ticker,date,AAPL,MSFT,NVDA,PLTR,SPY,TSLA
0,2023-10-31,169.113556,333.090363,40.758278,14.800000,408.794342,200.839996
1,2023-11-30,188.355331,374.042267,46.745087,20.049999,446.135223,240.080002
2,2023-12-31,190.913651,371.209106,49.499973,17.170000,466.503662,248.479996
3,2024-01-31,182.851913,392.472412,61.499634,16.090000,473.933441,187.289993
4,2024-02-29,179.461258,409.081238,79.076805,25.080000,498.666473,201.880005


Submit "historical_prices.tsv" in Gradescope.

## Part 2: Calculate Historical Asset Returns

Calculate the historical asset returns based on the price data that you previously collected.

In [6]:
# YOUR CHANGES HERE


import pandas as pd
import numpy as np

prices = pd.read_csv("historical_prices.tsv", sep="\t", parse_dates=["date"])
prices = prices.sort_values("date").reset_index(drop=True)

prices_idx = prices.set_index("date")

simple_rets = prices_idx.pct_change()

log_rets = np.log(prices_idx).diff()

out_simple = simple_rets.copy()
out_simple.index.name = "date"
out_simple = out_simple.reset_index()
out_simple.to_csv("historical_returns.tsv", sep="\t", index=False)

out_simple_no_nan = out_simple.dropna().reset_index(drop=True)
out_simple_no_nan.to_csv("historical_returns_no_nan.tsv", sep="\t", index=False)

print("Saved:")
print(" - historical_returns.tsv          (includes first NaN row)")
print(" - historical_returns_no_nan.tsv   (first row removed)")

display(out_simple.head(5))


Saved:
 - historical_returns.tsv          (includes first NaN row)
 - historical_returns_no_nan.tsv   (first row removed)


,date,AAPL,MSFT,NVDA,PLTR,SPY,TSLA
0,2023-10-31,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-11-30,0.113780,0.122945,0.146886,0.354730,0.091344,0.195379
2,2023-12-31,0.013582,-0.007574,0.058934,-0.143641,0.045655,0.034988
3,2024-01-31,-0.042227,0.057281,0.242418,-0.062900,0.015927,-0.246257
4,2024-02-29,-0.018543,0.042318,0.285809,0.558732,0.052187,0.077901


Save the data as a TSV file named "historical_returns.tsv" and include a header row with the column names "date" and the 6 stock ticker symbols.
Each row should have the date at the end of the month and the corresponding *relative* price changes.
For example, if the previous price was \$100 and the new price is \$110, the return value should be 0.10.
There should only be 23 rows of data in this file, since they are computed as the differences of 24 prices.

In [7]:
# YOUR CHANGES HERE


import pandas as pd

prices = pd.read_csv("historical_prices.tsv", sep="\t", parse_dates=["date"])
prices = prices.sort_values("date").reset_index(drop=True)

prices_idx = prices.set_index("date")

returns = prices_idx.pct_change()

returns = returns.dropna()

returns.index.name = "date"
returns_out = returns.reset_index()

returns_out.to_csv("historical_returns.tsv", sep="\t", index=False)

print("Saved historical_returns.tsv with shape:", returns_out.shape)
display(returns_out.head())


Saved historical_returns.tsv with shape: (23, 7)


,date,AAPL,MSFT,NVDA,PLTR,SPY,TSLA
0,2023-11-30,0.113780,0.122945,0.146886,0.354730,0.091344,0.195379
1,2023-12-31,0.013582,-0.007574,0.058934,-0.143641,0.045655,0.034988
2,2024-01-31,-0.042227,0.057281,0.242418,-0.062900,0.015927,-0.246257
3,2024-02-29,-0.018543,0.042318,0.285809,0.558732,0.052187,0.077901
4,2024-03-31,-0.051286,0.017116,0.142181,-0.082536,0.032702,-0.129235


Submit "historical_returns.tsv" in Gradescope.

## Part 3: Estimate Returns

Estimate the expected returns for each asset using the previously calculated return data.
Just compute the average (mean) return for each asset over your data set; do not use other estimators that have been mentioned.
This will serve as your estimate of expected return for each asset.

In [8]:
# YOUR CHANGES HERE


import pandas as pd

returns = pd.read_csv("historical_returns.tsv", sep="\t", parse_dates=["date"])

returns_only = returns.drop(columns=["date"])

expected_returns = returns_only.mean()

expected_df = expected_returns.reset_index()
expected_df.columns = ["Ticker", "ExpectedReturn"]

print("Estimated expected returns (mean monthly returns):")
display(expected_df)


Estimated expected returns (mean monthly returns):


,Ticker,ExpectedReturn
0,AAPL,0.016968
1,MSFT,0.019236
2,NVDA,0.069567
3,PLTR,0.121729
4,SPY,0.020758
5,TSLA,0.036103


Save the estimated returns in a TSV file named "estimated_returns.tsv" and include a header row with the column names "asset" and "estimated_return".

In [9]:
# YOUR CHANGES HERE


import pandas as pd

returns = pd.read_csv("historical_returns.tsv", sep="\t", parse_dates=["date"])

returns_only = returns.drop(columns=["date"])

expected_returns = returns_only.mean()

expected_df = expected_returns.reset_index()
expected_df.columns = ["asset", "estimated_return"]

expected_df.to_csv("estimated_returns.tsv", sep="\t", index=False)

print("Saved estimated_returns.tsv")
display(expected_df)


Saved estimated_returns.tsv


,asset,estimated_return
0,AAPL,0.016968
1,MSFT,0.019236
2,NVDA,0.069567
3,PLTR,0.121729
4,SPY,0.020758
5,TSLA,0.036103


Submit "estimated_returns.tsv" in Gradescope.

## Part 4: Estimate Risk

Estimate the covariance matrix for the asset returns to understand how the assets move together.

In [10]:
# YOUR CHANGES HERE


import pandas as pd

returns = pd.read_csv("historical_returns.tsv", sep="\t", parse_dates=["date"])

returns_only = returns.drop(columns=["date"])

cov_matrix = returns_only.cov()

print("Covariance matrix of monthly returns:")
display(cov_matrix)

cov_matrix.to_csv("covariance_matrix.tsv", sep="\t")
print("Saved covariance_matrix.tsv")


Covariance matrix of monthly returns:


,AAPL,MSFT,NVDA,PLTR,SPY,TSLA
AAPL,0.003623,0.000398,0.000768,0.001024,0.000784,0.003075
MSFT,0.000398,0.004022,0.005432,0.004540,0.001616,0.002690
NVDA,0.000768,0.005432,0.014287,0.003920,0.002673,-0.001848
PLTR,0.001024,0.004540,0.003920,0.038390,0.002387,0.017408
SPY,0.000784,0.001616,0.002673,0.002387,0.001225,0.001971
TSLA,0.003075,0.002690,-0.001848,0.017408,0.001971,0.024358


Saved covariance_matrix.tsv


Save the estimated covariances to a TSV file named "estimated_covariance.tsv".
The header row should have a blank column name followed by the names of the assets.
Each data row should start with the name of an asset for that row, and be followed by the individual covariances corresponding to that row and column's assets.
(This is the format of pandas's `to_csv` method with `sep="\t"` when used on a covariance matrix as computed in the examples.)

In [11]:
# YOUR CHANGES HERE


import pandas as pd

returns = pd.read_csv("historical_returns.tsv", sep="\t", parse_dates=["date"])

returns_only = returns.drop(columns=["date"])

cov_matrix = returns_only.cov()

cov_matrix.to_csv("estimated_covariance.tsv", sep="\t")

print("Saved estimated_covariance.tsv with shape:", cov_matrix.shape)
display(cov_matrix)


Saved estimated_covariance.tsv with shape: (6, 6)


,AAPL,MSFT,NVDA,PLTR,SPY,TSLA
AAPL,0.003623,0.000398,0.000768,0.001024,0.000784,0.003075
MSFT,0.000398,0.004022,0.005432,0.004540,0.001616,0.002690
NVDA,0.000768,0.005432,0.014287,0.003920,0.002673,-0.001848
PLTR,0.001024,0.004540,0.003920,0.038390,0.002387,0.017408
SPY,0.000784,0.001616,0.002673,0.002387,0.001225,0.001971
TSLA,0.003075,0.002690,-0.001848,0.017408,0.001971,0.024358


Submit "estimated_covariance.tsv" in Gradescope.

## Part 5: Construct the Maximum Return Portfolio

Compute the maximum return portfolio based on your previously estimated risks and returns.

In [12]:
# YOUR CHANGES HERE


import pandas as pd

expected = pd.read_csv("estimated_returns.tsv", sep="\t")

best_asset = expected.loc[expected["estimated_return"].idxmax(), "asset"]

weights = pd.DataFrame({
    "asset": expected["asset"],
    "weight": [1.0 if a == best_asset else 0.0 for a in expected["asset"]]
})

weights.to_csv("max_return_portfolio.tsv", sep="\t", index=False)

print("Maximum return portfolio invests fully in:", best_asset)
display(weights)


Maximum return portfolio invests fully in: PLTR


,asset,weight
0,AAPL,0.0
1,MSFT,0.0
2,NVDA,0.0
3,PLTR,1.0
4,SPY,0.0
5,TSLA,0.0


Save the maximum return portfolio in a TSV file named "maximum_return.tsv".
The header row should have two columns, "asset" and "allocation".
The allocation values should sum up to one.


In [13]:
# YOUR CHANGES HERE


import pandas as pd

expected = pd.read_csv("estimated_returns.tsv", sep="\t")

best_asset = expected.loc[expected["estimated_return"].idxmax(), "asset"]

allocations = pd.DataFrame({
    "asset": expected["asset"],
    "allocation": [1.0 if a == best_asset else 0.0 for a in expected["asset"]]
})

allocations.to_csv("maximum_return.tsv", sep="\t", index=False)

print("Saved maximum_return.tsv (all-in on:", best_asset, ")")
display(allocations)


Saved maximum_return.tsv (all-in on: PLTR )


,asset,allocation
0,AAPL,0.0
1,MSFT,0.0
2,NVDA,0.0
3,PLTR,1.0
4,SPY,0.0
5,TSLA,0.0


Submit "maximum_return.tsv" in Gradescope.

## Part 6: Construct the Minimum Risk Portfolio

Compute the minimum return portfolio based on your previously estimated risks and returns.

In [14]:
# YOUR CHANGES HERE


import pandas as pd
import numpy as np

returns = pd.read_csv("historical_returns.tsv", sep="\t", parse_dates=["date"])
assets = returns.columns[1:]   # skip 'date'

cov_matrix = returns[assets].cov().values

ones = np.ones(len(assets))

inv_cov = np.linalg.inv(cov_matrix)
w_gmv = inv_cov @ ones / (ones.T @ inv_cov @ ones)

min_risk_alloc = pd.DataFrame({
    "asset": assets,
    "allocation": w_gmv
})

min_risk_alloc["allocation"] /= min_risk_alloc["allocation"].sum()

min_risk_alloc.to_csv("minimum_risk.tsv", sep="\t", index=False)

print("Saved minimum_risk.tsv (weights sum to:", round(min_risk_alloc["allocation"].sum(), 6), ")")
display(min_risk_alloc)


Saved minimum_risk.tsv (weights sum to: 1.0 )


,asset,allocation
0,AAPL,0.114624
1,MSFT,0.093155
2,NVDA,-0.196720
3,PLTR,0.004267
4,SPY,1.079419
5,TSLA,-0.094745


Save the minimum risk portfolio in a TSV file named "minimum_risk.tsv".
The header row should have two columns, "asset" and "allocation".
The allocation values should sum up to one.


Submit "minimum_risk.tsv" in Gradescope.

In [3]:

import numpy as np
import pandas as pd
from scipy.optimize import minimize

rets = pd.read_csv("historical_returns.tsv", sep="\t", parse_dates=["date"])
assets = rets.columns[1:]  
Sigma = rets[assets].cov().values
n = len(assets)

def port_var(w):
    return float(w @ Sigma @ w)

cons = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1.0}]
bounds = [(0.0, 1.0) for _ in range(n)]
w0 = np.ones(n) / n  

res = minimize(port_var, w0, method='SLSQP', bounds=bounds, constraints=cons, options={'ftol':1e-12, 'maxiter':1000})
assert res.success, f"Optimization failed: {res.message}"

w_gmv_long = res.x
w_gmv_long = w_gmv_long / w_gmv_long.sum()  

out = pd.DataFrame({'asset': assets, 'allocation': w_gmv_long})
out.to_csv("minimum_risk.tsv", sep="\t", index=False)

print("Saved minimum_risk.tsv; weights sum to:", out['allocation'].sum().round(12))
display(out)


Saved minimum_risk.tsv; weights sum to: 1.0


,asset,allocation
0,AAPL,1.345442e-01
1,MSFT,3.040127e-17
2,NVDA,0.000000e+00
3,PLTR,0.000000e+00
4,SPY,8.654558e-01
5,TSLA,6.660778e-18


## Part 7: Build Efficient Frontier Portfolios

Compute 101 portfolios along the mean-variance efficient frontier with evenly spaced estimated returns.
The first portfolio should be the minimum risk portfolio from part 4, and the last portfolio should be the maximum return portfolio from part 3.
The estimated return of each portfolio should be higher than the previous by one percent of the difference between the first and last portfolios.
That is, the estimated return of the portfolios should be similar to `np.linspace(min_risk_return, max_return, 101)`.


In [ ]:
# YOUR CHANGES HERE


import numpy as np
import pandas as pd

er   = pd.read_csv("estimated_returns.tsv", sep="\t")          
rets = pd.read_csv("historical_returns.tsv",   sep="\t")       

assets = er["asset"].tolist()
mu     = er.set_index("asset")["estimated_return"].loc[assets].values  
Sigma  = rets[assets].cov().values                                     
n      = len(assets)
ones   = np.ones(n)

def weights_for_target_return(r):
    K = np.block([
        [2*Sigma,            mu.reshape(-1,1),  ones.reshape(-1,1)],
        [mu.reshape(1,-1),   np.zeros((1,1)),   np.zeros((1,1))   ],
        [ones.reshape(1,-1), np.zeros((1,1)),   np.zeros((1,1))   ],
    ])
    b = np.concatenate([np.zeros(n), [r], [1.0]])
    sol = np.linalg.solve(K, b)
    return sol[:n]  

invS = np.linalg.inv(Sigma)
A = ones @ invS @ ones
w_gmv = (invS @ ones) / A
min_risk_return = float(w_gmv @ mu)

max_return = float(mu.max())

target_returns = np.linspace(min_risk_return, max_return, 101)

W, variances, achieved = [], [], []
for r in target_returns:
    w = weights_for_target_return(r)
    W.append(w)
    variances.append(float(w @ Sigma @ w))
    achieved.append(float(w @ mu))

ef_df = pd.DataFrame(W, columns=assets)
ef_df.insert(0, "portfolio_variance", variances)
ef_df.insert(0, "target_return", target_returns)
ef_df["achieved_return"] = achieved  

assert np.allclose(ef_df[assets].sum(axis=1).values, 1.0, atol=1e-10)
assert np.allclose(ef_df["achieved_return"].values, ef_df["target_return"].values, atol=1e-10)

ef_df.to_csv("efficient_frontier.tsv", sep="\t", index=False)
print("Saved efficient_frontier.tsv")
print("First target return (min-risk):", round(min_risk_return, 6))
print("Last  target return (max-ret) :", round(max_return, 6))




Saved efficient_frontier.tsv
First target return (min-risk): 0.009557
Last  target return (max-ret) : 0.121729


Save the portfolios in a TSV file named "efficient_frontier.tsv".
The header row should have columns "index", "return", "risk", and all the asset tickers.
Each data row should have the portfolio index (0-100), the estimated return of the portfolio, the estimated standard deviation (not variance) of the portfolio, and all the asset allocations (which should sum to one).

In [ ]:
# YOUR CHANGES HERE


import numpy as np
import pandas as pd

er   = pd.read_csv("estimated_returns.tsv", sep="\t")         
rets = pd.read_csv("historical_returns.tsv", sep="\t")       
assets = er["asset"].tolist()
mu     = er.set_index("asset")["estimated_return"].loc[assets].values  
Sigma  = rets[assets].cov().values                                     
n      = len(assets)
ones   = np.ones(n)

def weights_for_target_return(r):
    K = np.block([
        [2*Sigma,            mu.reshape(-1,1),  ones.reshape(-1,1)],
        [mu.reshape(1,-1),   np.zeros((1,1)),   np.zeros((1,1))   ],
        [ones.reshape(1,-1), np.zeros((1,1)),   np.zeros((1,1))   ],
    ])
    b = np.concatenate([np.zeros(n), [r], [1.0]])
    sol = np.linalg.solve(K, b)
    return sol[:n] 

invS = np.linalg.inv(Sigma)
A = ones @ invS @ ones
w_gmv = (invS @ ones) / A
min_risk_return = float(w_gmv @ mu)

max_return = float(mu.max())

target_returns = np.linspace(min_risk_return, max_return, 101)

weights_list, returns_list, risks_list = [], [], []
for r in target_returns:
    w = weights_for_target_return(r)
    w = w / w.sum()
    weights_list.append(w)
    returns_list.append(float(w @ mu))
    risks_list.append(float(np.sqrt(w @ Sigma @ w)))  

ef = pd.DataFrame(weights_list, columns=assets)
ef.insert(0, "risk", risks_list)
ef.insert(0, "return", returns_list)
ef.insert(0, "index", np.arange(101, dtype=int))

assert np.allclose(ef[assets].sum(axis=1).values, 1.0, atol=1e-10)
assert len(ef) == 101
assert abs(ef.loc[0, "return"] - min_risk_return) < 1e-8
assert abs(ef.loc[100, "return"] - max_return) < 1e-8

ef.to_csv("efficient_frontier.tsv", sep="\t", index=False)

print("Saved efficient_frontier.tsv with shape:", ef.shape)
ef.head()


Saved efficient_frontier.tsv with shape: (101, 9)


,index,return,risk,AAPL,MSFT,NVDA,PLTR,SPY,TSLA
0,0,0.009557,0.029336,0.114624,0.093155,-0.196720,0.004267,1.079419,-0.094745
1,1,0.010679,0.029364,0.114439,0.073304,-0.184503,0.008980,1.081309,-0.093528
2,2,0.011801,0.029448,0.114253,0.053453,-0.172286,0.013693,1.083199,-0.092311
3,3,0.012922,0.029587,0.114068,0.033601,-0.160069,0.018405,1.085089,-0.091094
4,4,0.014044,0.029781,0.113882,0.013750,-0.147852,0.023118,1.086979,-0.089877


Submit "efficient_frontier.tsv" in Gradescope.

In [4]:
# --- Part 7: Long-only efficient frontier (101 portfolios) ---

import numpy as np
import pandas as pd
from scipy.optimize import minimize

# Inputs
er   = pd.read_csv("estimated_returns.tsv", sep="\t")  # columns: asset, estimated_return
rets = pd.read_csv("historical_returns.tsv", sep="\t") # columns: date, tickers

assets = er["asset"].tolist()
mu = er.set_index("asset")["estimated_return"].loc[assets].values
Sigma = rets[assets].cov().values
n = len(assets)

# Helper: variance objective & constraints
def port_var(w): return float(w @ Sigma @ w)

def solve_min_var_with_target(target_return):
    cons = [
        {'type': 'eq', 'fun': lambda w: np.sum(w) - 1.0},                # fully invested
        {'type': 'eq', 'fun': lambda w, tr=target_return: w @ mu - tr},  # target mean
    ]
    bounds = [(0.0, 1.0) for _ in range(n)]
    # Good starting point: tilt equal-weights toward the max-return asset
    w0 = np.zeros(n); w0[np.argmax(mu)] = 1.0
    res = minimize(port_var, w0, method='SLSQP', bounds=bounds, constraints=cons, options={'ftol':1e-12, 'maxiter':2000})
    if not res.success:
        # Sometimes the starting point causes trouble near boundaries; try equal-weights fallback
        w0 = np.ones(n)/n
        res = minimize(port_var, w0, method='SLSQP', bounds=bounds, constraints=cons, options={'ftol':1e-12, 'maxiter':2000})
        assert res.success, f"Frontier solve failed at target return {target_return}: {res.message}"
    w = res.x
    # Numerical tidy-up
    w[w < 0] = 0
    s = w.sum()
    if s <= 0:  # extreme numerical issue guard
        w = np.ones(n)/n
    else:
        w = w / s
    return w

# 1) Compute long-only GMV weights to get the starting return
def gmv_long_only(Sigma):
    def var(w): return float(w @ Sigma @ w)
    cons = [{'type':'eq', 'fun': lambda w: np.sum(w)-1.0}]
    bounds = [(0,1)]*n
    w0 = np.ones(n)/n
    res = minimize(var, w0, method='SLSQP', bounds=bounds, constraints=cons, options={'ftol':1e-12, 'maxiter':1000})
    assert res.success, res.message
    w = res.x; w = w / w.sum()
    return w

w_gmv_lo = gmv_long_only(Sigma)
min_risk_return = float(w_gmv_lo @ mu)

# 2) Max single-asset expected return (end of grid); last portfolio must be one-hot on this asset
imax = int(np.argmax(mu))
max_return = float(mu[imax])
unit_max = np.zeros(n); unit_max[imax] = 1.0

# 3) Build target return grid and solve frontier
target_returns = np.linspace(min_risk_return, max_return, 101)
rows = []
for i, tr in enumerate(target_returns):
    # For the very last point, we *know* the long-only minimum-variance solution is all-in on the max asset
    if i == len(target_returns) - 1:
        w = unit_max.copy()
    else:
        w = solve_min_var_with_target(tr)
    pr = float(w @ mu)
    risk = float(np.sqrt(w @ Sigma @ w))
    rows.append([i, pr, risk, *w])

ef = pd.DataFrame(rows, columns=["index", "return", "risk", *assets])

# Sanity: allocations sum to 1; first return matches GMV long-only; last weights one-hot on max asset
assert np.allclose(ef[assets].sum(axis=1).values, 1.0, atol=1e-8)
assert abs(ef.loc[0, "return"] - min_risk_return) < 1e-8
assert np.allclose(ef.loc[100, assets].values, unit_max, atol=1e-8)

# Save exactly as required
ef.to_csv("efficient_frontier.tsv", sep="\t", index=False)

print("Saved efficient_frontier.tsv. First/last returns:",
      round(ef.loc[0,'return'], 8), "→", round(ef.loc[100,'return'], 8))
display(ef.head())
display(ef.tail())


Saved efficient_frontier.tsv. First/last returns: 0.02024802 → 0.12172875


,index,return,risk,AAPL,MSFT,NVDA,PLTR,SPY,TSLA
0,0,0.020248,0.034146,0.134544,0.000000e+00,0.000000e+00,1.322707e-17,0.865456,0.000000e+00
1,1,0.021263,0.034493,0.121403,8.230517e-17,3.268304e-17,9.557215e-03,0.869040,5.120544e-17
2,2,0.022278,0.034944,0.120213,0.000000e+00,1.510006e-17,1.956307e-02,0.860224,2.389708e-17
3,3,0.023292,0.035488,0.119024,5.701274e-17,0.000000e+00,2.956892e-02,0.851407,0.000000e+00
4,4,0.024307,0.036122,0.117834,0.000000e+00,5.067875e-18,3.957478e-02,0.842591,0.000000e+00


,index,return,risk,AAPL,MSFT,NVDA,PLTR,SPY,TSLA
96,96,0.117670,0.182473,0.000000e+00,0.000000e+00,0.077819,0.922181,3.498325e-17,9.718728e-19
97,97,0.118684,0.185793,2.754100e-17,2.427837e-23,0.058365,0.941635,2.450513e-18,1.091544e-17
98,98,0.119699,0.189143,8.104629e-19,4.582221e-19,0.038910,0.961090,1.607529e-25,3.082472e-19
99,99,0.120714,0.192524,0.000000e+00,0.000000e+00,0.019455,0.980545,2.220047e-17,0.000000e+00
100,100,0.121729,0.195933,0.000000e+00,0.000000e+00,0.000000,1.000000,0.000000e+00,0.000000e+00


## Part 8: Check Maximum Return Portfolio Stability

Check the stability of the maximum return portfolio by resampling the estimated risk/return model.

Repeat 1000 times -
1. Use `np.random.multivariate_normal` to generate 23 return samples using your previously estimated risks and returns.
2. Estimate the return of each asset using that resampled return history.
3. Check which asset had the highest return in those resampled estimates.

This procedure is a reduced and simplified version of the Michaud resampled efficient frontier procedure that takes uncertainty in the risk model into account.

In [ ]:
# YOUR CHANGES HERE


import numpy as np
import pandas as pd

er    = pd.read_csv("estimated_returns.tsv", sep="\t")          
rets  = pd.read_csv("historical_returns.tsv",  sep="\t")         

assets = er["asset"].tolist()
mu     = er.set_index("asset")["estimated_return"].loc[assets].values    
Sigma  = rets[assets].cov().values                                       
T      = rets.shape[0]                                                   
n_iter = 1000
rng    = np.random.default_rng(42)  

eigvals = np.linalg.eigvalsh(Sigma)
if eigvals.min() < 0:
    Sigma = Sigma + np.eye(len(assets)) * ( -eigvals.min() + 1e-12 )

winners = []

for _ in range(n_iter):
    sample = rng.multivariate_normal(mean=mu, cov=Sigma, size=T)  # shape (T, n_assets)
    est_means = sample.mean(axis=0)
    winners.append(assets[int(np.argmax(est_means))])

counts = pd.Series(winners, name="count").value_counts().reindex(assets, fill_value=0)
props  = counts / n_iter

stability = pd.DataFrame({
    "asset": assets,
    "wins": counts.values,
    "proportion": props.values
})

stability.to_csv("max_return_stability.tsv", sep="\t", index=False)
pd.DataFrame({"iteration": np.arange(n_iter), "winner": winners}) \
  .to_csv("max_return_winners_by_iter.tsv", sep="\t", index=False)

print("Saved:")
print(" - max_return_stability.tsv   (wins and proportions by asset)")
print(" - max_return_winners_by_iter.tsv  (winner asset for each resample)")
display(stability)


Saved:
 - max_return_stability.tsv   (wins and proportions by asset)
 - max_return_winners_by_iter.tsv  (winner asset for each resample)


,asset,wins,proportion
0,AAPL,1,0.001
1,MSFT,0,0.000
2,NVDA,123,0.123
3,PLTR,875,0.875
4,SPY,0,0.000
5,TSLA,1,0.001


Save a file "max_return_probabilities.tsv" with the distribution of highest return assets.
The header row should have columns "asset" and "probability".
There should be a data row for each asset and its sample probability of having the highest return based on those 1000 resampled estimates.


In [19]:
# YOUR CHANGES HERE


import numpy as np
import pandas as pd

er    = pd.read_csv("estimated_returns.tsv", sep="\t")
rets  = pd.read_csv("historical_returns.tsv", sep="\t")

assets = er["asset"].tolist()
mu     = er.set_index("asset")["estimated_return"].loc[assets].values
Sigma  = rets[assets].cov().values
T      = rets.shape[0]      
n_iter = 1000
rng    = np.random.default_rng(42)

eigvals = np.linalg.eigvalsh(Sigma)
if eigvals.min() < 0:
    Sigma = Sigma + np.eye(len(assets)) * (-eigvals.min() + 1e-12)

winners = []

for _ in range(n_iter):
    sample = rng.multivariate_normal(mean=mu, cov=Sigma, size=T)
    est_means = sample.mean(axis=0)
    winners.append(assets[int(np.argmax(est_means))])

counts = pd.Series(winners).value_counts().reindex(assets, fill_value=0)
probs  = counts / n_iter

prob_df = pd.DataFrame({
    "asset": assets,
    "probability": probs.values
})

prob_df.to_csv("max_return_probabilities.tsv", sep="\t", index=False)

print("Saved max_return_probabilities.tsv")
display(prob_df)


Saved max_return_probabilities.tsv


,asset,probability
0,AAPL,0.001
1,MSFT,0.000
2,NVDA,0.123
3,PLTR,0.875
4,SPY,0.000
5,TSLA,0.001


Submit "max_return_probabilities.tsv" in Gradescope.

## Part 9: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgements.txt" in Gradescope.

In [6]:
acknowledgments_text = "My dad helped me with this"

with open("acknowledgments.txt", "w") as f:
    f.write(acknowledgments_text)

print("Saved acknowledgments.txt")

Saved acknowledgments.txt


## Part 10: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

Submit "project.ipynb" in Gradescope.